In [1]:
import os
from pathlib import Path
import sys
curdir = Path(os.getcwd())
sys.path.append(str(curdir.parent.absolute()))
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from src.utils.data import read_fasta
from src.data.datasets import ProteinDataset

In [2]:
train = read_fasta('../data/swissprot/proteinfer_splits/random/train_GO.fasta')
val = read_fasta('../data/swissprot/proteinfer_splits/random/dev_GO.fasta')
test = read_fasta('../data/swissprot/proteinfer_splits/random/test_GO.fasta')

In [ ]:
train = [(k[0],j," ".join(k[1:])) for j,k in train]
test = [(k[0],j," ".join(k[1:])) for j,k in test]
val = [(k[0],j," ".join(k[1:])) for j,k in val]

df = train + val + test

In [4]:
df = pd.DataFrame(df,columns=['id','sequence','labels'])

In [5]:
num_sequences = len(df)
print('number of sequences:',num_sequences)

number of sequences: 522607


In [6]:
labels = Counter()

vocab = set()
amino_freq = Counter()
for idx,row in df.iterrows():
    sequence = row['sequence']
    row_labels = row['labels']
    aa_list = list(sequence)
    if row_labels =='':
        print(row['id'],row['labels'])
    vocab.update(aa_list)
    amino_freq.update(aa_list)
    labels.update(row_labels.split(" "))

In [7]:
print('# GO Terms:',len(labels.keys()))

# GO Terms: 32102


In [8]:
print('GO Terms distribution')
pd.Series(labels.values()).describe()

GO Terms distribution


count     32102.000000
mean        777.250545
std        9114.786603
min           1.000000
25%           4.000000
50%          17.000000
75%          84.000000
max      462356.000000
dtype: float64

In [9]:
print('Sequence length distribution')

df['sequence'].apply(len).describe()

Sequence length distribution


count    522607.000000
mean        368.042215
std         334.721845
min           2.000000
25%         179.000000
50%         303.000000
75%         456.000000
max       35213.000000
Name: sequence, dtype: float64

In [2]:
PD = ProteinDataset(data_path='../data/swissprot/proteinfer_splits/random/train_GO.fasta',
                    sequence_vocabulary_path='../data/vocabularies/amino_acid_vocab.json',
                    label_vocabulary_path='../data/vocabularies/GO_label_vocab.json')

In [3]:
test_seqs_preds = pd.read_csv('../proteinfer/hemoglobin_predictions.tsv',sep='\t')

In [4]:
PD.data = [i for i in PD.data if i[1][0] in ['P69891','Q7AP54']]

In [5]:
from sklearn.metrics import precision_score,recall_score

In [6]:
len(PD),PD.get_max_seq_len()

(2, 569)

In [7]:
from src.models.protein_encoders import ProteInfer
import torch
model = ProteInfer(num_labels=32102,
                      input_channels=20,
                      output_channels=1100,
                      kernel_size=9,
                      activation=torch.nn.ReLU,
                      dilation_base=3,
                      num_resnet_blocks=5,
                      bottleneck_factor=0.5)

from src.utils.proteinfer import transfer_tf_weights_to_torch
transfer_tf_weights_to_torch(model,'../models/proteinfer/GO_model_weights.pkl')

model.to('cuda:0')

Pytorch num variables: 74
Tensorflow num variables: 74


conv1.weight:torch.Size([1100, 20, 9]) <--> inferrer/conv1d/kernel:0:(1100, 20, 9)
conv1.bias:torch.Size([1100]) <--> inferrer/conv1d/bias:0:(1100,)
resnet_blocks.0.bn_activation_1.0.weight:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/gamma:0:(1100,)
resnet_blocks.0.bn_activation_1.0.bias:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/beta:0:(1100,)
resnet_blocks.0.bn_activation_1.0.running_mean:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/moving_mean:0:(1100,)
resnet_blocks.0.bn_activation_1.0.running_var:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/moving_variance:0:(1100,)
resnet_blocks.0.bn_activation_1.0.num_batches_tracked:torch.Size([]) <--> inferrer/residual_block_0/batch_normalization/num_batches_tracked:0:()
resnet_blocks.0.masked_conv1.weight:torch.Size([550, 1100, 9]) <--> inferrer/residual_block_0/conv1d/kernel:0:(550, 1

ProteInfer(
  (conv1): MaskedConv1D(20, 1100, kernel_size=(9,), stride=(1,), padding=same)
  (resnet_blocks): ModuleList(
    (0): Residual(
      (bn_activation_1): Sequential(
        (0): BatchNorm1d(1100, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv1): MaskedConv1D(1100, 550, kernel_size=(9,), stride=(1,), padding=same)
      (bn_activation_2): Sequential(
        (0): BatchNorm1d(550, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv2): MaskedConv1D(550, 1100, kernel_size=(1,), stride=(1,), padding=same)
    )
    (1): Residual(
      (bn_activation_1): Sequential(
        (0): BatchNorm1d(1100, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv1): MaskedConv1D(1100, 550, kernel_size=(9,), stride=(1,), padding=same, dilation=(3,))
      (bn_activation_2): Sequential(
        (0): BatchNorm1d(55

In [8]:
from torch.utils.data import DataLoader
from src.data.collators import collate_variable_sequence_length
PD_loader = DataLoader(dataset=PD,
                          batch_size=2,
                          shuffle=False,
                          num_workers=2,
                          collate_fn=collate_variable_sequence_length)

In [10]:
with torch.no_grad():
    for i,l,s in PD_loader:
        print(i.shape,s.shape,l.shape)
        i=torch.cat((i[0],torch.zeros((20,0))),dim=1).unsqueeze(0)
        logits = model(i.type(torch.FloatTensor).to('cuda:0'),s.to('cuda:0'))
        probas = torch.sigmoid(logits)
        print(probas)
        above_th = (probas>0.5)*1
        print(above_th.sum(axis=1))

torch.Size([2, 20, 569]) torch.Size([2, 32102]) torch.Size([2])
input_seq torch.Size([1, 20, 569]) tensor(0.0129, device='cuda:0') tensor(147., device='cuda:0')
input_seq_len torch.Size([2, 32102]) tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')


RuntimeError: The size of tensor a (20) must match the size of tensor b (32102) at non-singleton dimension 2

In [9]:
with torch.no_grad():
    for i,s,l in PD_loader:
        logits = model(i.type(torch.FloatTensor).to('cuda:0'),s.to('cuda:0'))
        probas = torch.sigmoid(logits)
        above_th = (probas>0.5)*1


input_seq torch.Size([2, 20, 569]) tensor(0.0315, device='cuda:0') tensor(716., device='cuda:0')
input_seq_len torch.Size([2]) tensor([147, 569], device='cuda:0')
input_to_conv torch.Size([2, 20, 569]) tensor(0.0315, device='cuda:0') tensor(716., device='cuda:0')
output_of_conv torch.Size([2, 1100, 569]) tensor(-1.1537, device='cuda:0') tensor(-1444248.5000, device='cuda:0')
tensor([[[-0.7153, -0.6415, -0.3653,  ...,  0.0000,  0.0000,  0.0000],
         [-2.3647,  0.4084, -2.2052,  ...,  0.0000,  0.0000,  0.0000],
         [-0.2829, -2.3705,  0.3380,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-1.0497, -2.2400, -1.3620,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.3808, -1.9959, -0.6329,  ...,  0.0000,  0.0000,  0.0000],
         [-1.4631, -1.0661, -2.8533,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.9580, -0.7764,  0.2500,  ..., -0.0622, -1.5976, -1.4675],
         [-0.0781,  1.4956, -1.2454,  ..., -2.4412, -1.7659, -1.5335],
         [-0.6667,  0.2899,  0.2071,  

In [28]:
import torch

def set_padding_to_sentinel(padded_representations, sequence_lengths, sentinel):
    """
    Set the padding values in the input tensor to the sentinel value.
    
    Parameters:
        padded_representations (torch.Tensor): The input tensor of shape (batch_size, dim, max_sequence_length)
        sequence_lengths (torch.Tensor): 1D tensor containing original sequence lengths for each sequence in the batch
        sentinel (float): The value to set the padding to
        
    Returns:
        torch.Tensor: Tensor with padding values set to sentinel
    """
    
    # Get the shape of the input tensor
    batch_size, dim, max_sequence_length = padded_representations.shape
    
    # Create a mask that identifies padding
    mask = torch.arange(max_sequence_length).expand(batch_size, max_sequence_length) >= sequence_lengths.unsqueeze(1)
    
    # Expand the mask to cover the 'dim' dimension
    mask = mask.unsqueeze(1).expand(-1, dim, -1)
    
    # Use the mask to set the padding values to sentinel
    padded_representations[mask] = sentinel
    
    return padded_representations

# Example usage:
batch_size = 3
dim = 4
max_sequence_length = 6
padded_representations = torch.randn(batch_size, dim, max_sequence_length)
sequence_lengths = torch.tensor([4, 5, 6])
sentinel = -999

print("Before:")
print(padded_representations)

print("After:")
print(set_padding_to_sentinel(padded_representations, sequence_lengths, sentinel))


Before:
tensor([[[ 1.0752,  0.5794,  1.2520, -2.1861, -0.2771, -0.0278],
         [-1.1708, -1.1192,  1.7641, -1.4485,  0.2079,  0.1950],
         [-0.1735,  0.4402,  0.4596,  0.0405, -0.7241, -1.0551],
         [ 0.7670,  0.4493,  0.6640, -1.3435,  1.2225,  1.1037]],

        [[-0.7285, -0.1041, -2.1080, -0.0435,  1.3493, -1.2803],
         [-0.3031, -0.3673,  1.9882, -0.7333, -1.7588, -0.4458],
         [ 0.5464, -1.3600,  1.8604,  1.1871,  1.8073, -0.2755],
         [-1.2164,  0.0813, -1.8185, -0.3479, -0.7302,  0.7456]],

        [[ 2.2673,  0.0968,  0.2001,  0.2980, -0.9458, -1.0923],
         [-0.5151, -1.9390,  1.6619, -0.1635,  1.0856, -1.2883],
         [ 0.0744,  1.0954,  0.6909, -1.9535, -0.0256, -0.5458],
         [ 1.5307, -0.1504, -0.6249, -0.0658,  1.0999,  0.8504]]])
After:
tensor([[[ 1.0752e+00,  5.7935e-01,  1.2520e+00, -2.1861e+00, -9.9900e+02,
          -9.9900e+02],
         [-1.1708e+00, -1.1192e+00,  1.7641e+00, -1.4485e+00, -9.9900e+02,
          -9.9900e+02],
 

In [20]:
seq_mask = torch.arange(padded_representations.size(1), device=sequence_lengths.device)


In [24]:
padded_representations.shape

torch.Size([2, 600, 10])

In [22]:
sequence_lengths

tensor([147, 569], device='cuda:0')

In [21]:
seq_mask

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [15]:
print("Sequence mask:", seq_mask)


NameError: name 'seq_mask' is not defined

In [14]:
output

tensor([[[0.3871, 0.1797, 0.5165,  ..., 0.6133, 0.4819, 0.7436],
         [0.0933, 0.9330, 0.2361,  ..., 0.8017, 0.6164, 0.3758],
         [0.5702, 0.9327, 0.9981,  ..., 0.7602, 0.3556, 0.7660],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.2828, 0.1742, 0.1041,  ..., 0.6913, 0.9794, 0.7480],
         [0.3496, 0.2316, 0.5343,  ..., 0.2502, 0.8556, 0.7232],
         [0.4305, 0.5330, 0.6826,  ..., 0.8251, 0.3763, 0.1209],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       device='cuda:0')

In [21]:
with torch.no_grad():
    for i,s,l in PD_loader:
        logits = model(i.type(torch.FloatTensor).to('cuda:0'),s.to('cuda:0'))
        probas = torch.sigmoid(logits)
        above_th = (probas>0.5)*1


first_conv torch.Size([2, 1100, 569]) tensor(-0.4546, device='cuda:0')
torch.Size([2, 550, 569]) tensor(-4.2635, device='cuda:0')
torch.Size([2, 550, 569]) tensor(-19.1667, device='cuda:0')
torch.Size([2, 550, 569]) tensor(-26.5519, device='cuda:0')
torch.Size([2, 550, 569]) tensor(-21.3674, device='cuda:0')
torch.Size([2, 550, 569]) tensor(-9.5727, device='cuda:0')


In [2]:
import torch
from src.models.protein_encoders import Residual,ProteInfer
i=torch.ones((8,20,100))
#r=Residual(input_channels=20,kernel_size=9,dilation=9,bottleneck_factor=0.5,activation = torch.nn.ReLU)
prot_enc = ProteInfer(num_labels=32102,input_channels=20,output_channels=1100,kernel_size=9,activation=torch.nn.ReLU,dilation_base=3,num_resnet_blocks=5,bottleneck_factor=0.5)
seqs_lengths = torch.tensor([80,20,5,100,95,80,20,5])
o=prot_enc(i,seqs_lengths)
o.shape

TypeError: conv1d(): argument 'padding' (position 5) must be tuple of ints, not str

In [3]:
from src.utils.proteinfer import transfer_tf_weights_to_torch
transfer_tf_weights_to_torch(prot_enc,'../models/proteinfer/GO_model_weights.pkl')

Pytorch num variables: 74
Tensorflow num variables: 74


conv1.weight:torch.Size([1100, 20, 9]) <--> inferrer/conv1d/kernel:0:(1100, 20, 9)
conv1.bias:torch.Size([1100]) <--> inferrer/conv1d/bias:0:(1100,)
resnet_blocks.0.bn_activation_1.0.weight:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/gamma:0:(1100,)
resnet_blocks.0.bn_activation_1.0.bias:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/beta:0:(1100,)
resnet_blocks.0.bn_activation_1.0.running_mean:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/moving_mean:0:(1100,)
resnet_blocks.0.bn_activation_1.0.running_var:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/moving_variance:0:(1100,)
resnet_blocks.0.bn_activation_1.0.num_batches_tracked:torch.Size([]) <--> inferrer/residual_block_0/batch_normalization/num_batches_tracked:0:()
resnet_blocks.0.masked_conv1.weight:torch.Size([550, 1100, 9]) <--> inferrer/residual_block_0/conv1d/kernel:0:(550, 1

In [4]:
prot_enc

ProteInfer(
  (conv1): MaskedConv1D(20, 1100, kernel_size=(9,), stride=(1,), padding=same)
  (resnet_blocks): ModuleList(
    (0): Residual(
      (bn_activation_1): Sequential(
        (0): BatchNorm1d(1100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv1): MaskedConv1D(1100, 550, kernel_size=(9,), stride=(1,), padding=same)
      (bn_activation_2): Sequential(
        (0): BatchNorm1d(550, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv2): MaskedConv1D(550, 1100, kernel_size=(1,), stride=(1,), padding=same)
    )
    (1): Residual(
      (bn_activation_1): Sequential(
        (0): BatchNorm1d(1100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv1): MaskedConv1D(1100, 550, kernel_size=(9,), stride=(1,), padding=same, dilation=(3,))
      (bn_activation_2): Sequential(
        (0): BatchNorm1d(550, 